# Taller 2: Métricas de Desempeño
---

Este taller busca evaluar conceptos de modelamiento y la implementación de métricas de desempeño para la evaluación de modelos de Machine Learning supervisados.

Librerías:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from numpy.typing import ArrayLike
from typing import Tuple
from IPython.display import display
import seaborn as sns

## **1. Carga de Datos**
---

En este caso trabajáremos con el conjunto de datos [Heart Attack Analysis & Prediction Dataset](https://www.kaggle.com/datasets/rashikrahmanpritom/heart-attack-analysis-prediction-dataset?select=heart.csv) de Kaggle, vamos a descargarlo:

In [ ]:
!wget 'https://drive.google.com/uc?export=view&id=1djX2CPMY-O_vskg9ey326auY8Yho415v' -O heart_failure.zip
!unzip heart_failure.zip

--2023-04-24 13:07:54--  https://drive.google.com/uc?export=view&id=1djX2CPMY-O_vskg9ey326auY8Yho415v
Resolving drive.google.com (drive.google.com)... 74.125.204.138, 74.125.204.100, 74.125.204.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.204.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-4g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8526sjo83eu0kpmqi7mgekaiebt7ftj7/1682341650000/16848862265445619282/*/1djX2CPMY-O_vskg9ey326auY8Yho415v?e=view&uuid=b1065e73-67c5-438f-be63-86640665183f [following]
--2023-04-24 13:07:55--  https://doc-10-4g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8526sjo83eu0kpmqi7mgekaiebt7ftj7/1682341650000/16848862265445619282/*/1djX2CPMY-O_vskg9ey326auY8Yho415v?e=view&uuid=b1065e73-67c5-438f-be63-86640665183f
Resolving doc-10-4g-docs.googleusercontent.com (doc-10-4g-docs.googleusercontent.com)... 64.233.188.132, 2404:6800:4008:c0

Cargamos el conjunto de datos:

In [ ]:
data = pd.read_csv("heart.csv")
display(data.head())

## **2. Análisis Exploratorio**
---

En este punto deberá explorar y entender el conjunto de datos y sus elementos.

- ¿Cuántas columnas tiene el conjunto de datos, qué significa cada una?

**INGRESE SU RESPUESTA ACÁ**

**Respuesta =** En la base de datos tenemos un total de 14 columnas, las cuales incluyen información del paciente como **age:** que corresponde a la edad del paciente, **sex:** sexo del paciente, **cp:** que identifica el tipo del dolor torácico, tenemos 4, angina típica, angina atípica, dolor no aginoso y dolor asintomático. Por otro lado tenemos **trtbps:** el cual es la presión arterial en reposo, este se mide en mm y Hg, **chol:** que coincide con el coresterol por BMI en mg/dl, **fbs:** variable que se ajusta al azucar en la sangre en ayunas, este se mide en 120 mg/dl, si este cumple retornará 1, si no lo hace, este retornará 0. **rest_ecg:** son basicamente los resultados electrocardiográficos en reposo, este retornará 0 si esta en estado normal, pero si ste presenta problemas en la onda ST-T retornará 1, así mismo retornará 2 para mostrar la hipertrofia ventricular izquierda probable o definida a partir de los criterios de estres. Tenemos **thalach:** la cual respresenta la frecuencia cárdiaca máxima alcanzada, así mismo esta **exng:** variable que responde a la angina inducida por el ejercicio, **oldpeak:** es una medida del descenso del segmento ST después del ejercicio en relación con el nivel de reposo. Es un indicador de la gravedad de la enfermedad cardíaca, **slp:** es una variable categórica que describe la pendiente del segmento ST en el pico del ejercicio, **caa:** es una variable categórica que indica el número de vasos principales coloreados por flourosopía, **thal:** es una variable categórica que indica el resultado del estrés cardíaco según la escala de Thal, **output:** es una variable categórica binaria que indica la presencia o ausencia de enfermedad cardíaca.

In [ ]:
data.isnull().sum()

Implemente una función que permita determinar si una columna es continúa o categórica (puede guiarse por el número de elementos únicos que tenga cada columna):

In [ ]:
def continuous_or_categoric(data: pd.DataFrame, column: str) -> bool:

    # Comprueba si la columna es de tipo "Object" o "boolean".
    if data[column].dtype in ['object', 'bool']:
        return False

    # Validar si la columna está clasificada como "datetime".
    if pd.api.types.is_datetime64_any_dtype(data[column].dtype):
        return True

    # Asegurarse de que la columna tenga el tipo de datos "integer" o "float".
    if pd.api.types.is_numeric_dtype(data[column].dtype):
        # Verificar si la columna tiene una cardinalidad (número de valores distintos) menor a 20.
        if data[column].nunique() <= 20:
            return False
        else:
            return True

    # En situaciones donde no se pueda determinar el tipo de la columna, devolver "desconocida".
    return 'desconocida'

Use la siguiente celda para probar su código (puede cambiar el nombre de la variable).

In [ ]:
display(continuous_or_categoric(data, "age"))

- ¿Cómo es la distribución de las columnas?

Para esto debe implementar la función `show_distribution` la cual debe graficar un diagrama de tipo [kernel density estimation](https://seaborn.pydata.org/generated/seaborn.kdeplot.html) para las variables continuas y un diagrama de barras para las variables categóricas:

In [ ]:
def show_distribution(data: pd.DataFrame, column: str) -> plt.Figure:
    if continuous_or_categoric(data, column) == 'desconocida':
        raise ValueError(f'Tipo de datos desconocido para la columna {column}.')

    fig, ax = plt.subplots()
    if continuous_or_categoric(data, column):
        sns.kdeplot(data=data, x=column, ax=ax)
    else:
        sns.countplot(data=data, x=column, ax=ax)

    return fig

Utilice la siguiente celda para probar su código:

In [ ]:
fig = show_distribution(data, "age")
fig.show()

- ¿Cuáles son las variables independientes y cuál es la variable dependiente?

Para esto debe implementar la función `target_variable` la cual debe separar las columnas que se tomarán como variables independientes de la columna objetivo.

In [ ]:
def target_variable(data: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:

    features, labels = data.iloc[:, :-1], data.iloc[:, -1]
    return features, labels

Utilice la siguiente celda para probar su código:

In [ ]:
features, labels = target_variable(data)

## **3. Preprocesamiento**
---

En este punto debe implementar una función para preprocesar el conjunto de datos. debe aplicar una transformación de tipo `min_max` sobre los datos (cada columna debe estar entre 0 y 1):

$$
x_{minmax} = \frac{x - \text{min}(x)}{\text{max}(x) - \text{min}(x)}
$$

Puede utilizar la clase `MinMaxScaler` de `sklearn`. Adicionalmente, debe convertir las etiquetas a un arreglo de `numpy`.

In [ ]:
def preprocess(features: pd.DataFrame, labels: pd.Series) -> Tuple[ArrayLike, ArrayLike]:

    # MinMaxScaler
    scaler = MinMaxScaler()
    features_p, labels_p = scaler.fit_transform(features), np.array(labels)
    return features_p, labels_p

Utilice la siguiente celda para probar su código:

In [ ]:
features_p, labels_p = preprocess(features, labels)

## **4. Modelamiento**
---

Para entrenar el modelo, dividimos el conjunto de datos en las particiones de entrenamiento y prueba:

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(
        features_p, labels_p, stratify=labels_p, test_size=0.3
        )

Para el modelo necesitamos optimizar una función de pérdida conocida como la entropía binaria cruzada':

$$
\mathcal{L} = - \frac{1}{N} \sum_{i=1} ^ N y_i \log{\tilde{y}_i} + (1 - y_i) \log{(1 - \tilde{y}_i)}
$$

Debe implementar esta función:

In [ ]:
def binary_crossentropy(y: ArrayLike, y_pred: ArrayLike) -> ArrayLike:

    y = np.array(y)
    y_pred = np.array(y_pred)

    loss = -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))

    return loss

Utilice las siguientes celdas para probar su código:

In [ ]:
y = np.array([0., 0])
y_pred = np.array([0.01, 0.01])
# Debe dar un valor bajo al ser valores parecidos
print(binary_crossentropy(y, y_pred))

In [ ]:
y = np.array([1., 1.])
y_pred = np.array([0.99, 0.99])
# Debe dar un valor bajo al ser valores parecidos
print(binary_crossentropy(y, y_pred))

In [ ]:
y = np.array([0., 0.])
y_pred = np.array([0.99, 0.99])
# Debe dar un valor alto al ser valores distintos
print(binary_crossentropy(y, y_pred))

In [ ]:
y = np.array([1., 1.])
y_pred = np.array([0.01, 0.01])
# Debe dar un valor alto al ser valores distintos
print(binary_crossentropy(y, y_pred))

Ahora, debe implementar un modelo de regresión logistica:

$$
\tilde{\mathbf{y}} = \frac{1}{1 + e^{\mathbf{X}\cdot\mathbf{w}}}
$$

Donde $\mathbf{X}$ corresponde a la matriz de características de entrenamiento, $\mathbf{w}$ es el vector de parámetros y $\tilde{\mathbf{y}}$ es la estimación del modelo.

> **Nota**: recuerde agregar una columna de unos para considerar el intercepto. El entrenamiento del modelo debe realizarse optimizando la entropía binaria cruzada, puede utilizar la función `optimize` de `scipy` para esto.

In [ ]:
class LogisticRegression(BaseEstimator, ClassifierMixin):

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X: ArrayLike, y: ArrayLike) -> "LogisticRegression":

        intercept = np.ones((X.shape[0], 1))
        np.concatenate((intercept, X), axis=1)
        self.w = np.zeros(X.shape[1])

        loss_fn = lambda w: binary_crossentropy(y, self.sigmoid(np.dot(X, w)))

        self.w = optimize.minimize(loss_fn, self.w).x
        return self

    def predict_proba(self, X: ArrayLike) -> ArrayLike:

        probs = self.sigmoid(np.dot(X, self.w))
        return np.vstack([1 - probs, probs]).T

    def predict(self, X: ArrayLike) -> ArrayLike:

        y_pred = (self.predict_proba(X)[:, 1] >= 0.5).astype(int)
        return y_pred

Entrenamos el modelo:

In [ ]:
model = LogisticRegression().fit(features_train, labels_train)

## **5. Evaluación**
---

Obtenga las predicciones del modelo:

In [ ]:
y_pred = model.predict(features_test)
print(y_pred)

- ¿Cuánto da el accuracy del modelo?

In [ ]:
accuracy = accuracy_score(labels_test, y_pred)
print(accuracy)

- ¿Cuánto da la precisión del modelo?

In [ ]:
print(precision_score(labels_test,y_pred))

- ¿Cuánto da el recall del modelo?

In [ ]:
recall = recall_score(labels_test, y_pred)
print(recall)

- ¿Cuánto da el f1 del modelo?

In [ ]:
print(f1_score(labels_test, y_pred))

- ¿Qué puede concluir de las métricas y del modelo?

La métrica de **accuracy** es una medida importante para evaluar el rendimiento general de un modelo, ya que indica qué tan precisa es su predicción en términos generales. Sin embargo, si la distribución de las clases es desigual, la precisión no será suficiente para evaluar el rendimiento del modelo, ya que el modelo podría simplemente predecir siempre la clase mayoritaria para lograr una alta precisión, en lugar de hacer predicciones precisas para todas las clases.

La métrica de **precision** es especialmente útil cuando los falsos positivos son perjudiciales, ya que indica que el modelo hace muy pocas predicciones falsas positivas.

La métrica de **recall** es importante cuando se necesitan detectar la mayoría de los casos positivos, especialmente en casos donde los falsos negativos son perjudiciales.

La puntuación F1 es una métrica que combina tanto el recall como la precision, lo que la convierte en una buena opción si se desea equilibrar ambas métricas.

En este estudio, se utilizó un modelo de regresión logística que funciona razonablemente bien en el dataset utilizado para evaluar su rendimiento.



